In [1]:
import sqlite3
# import ibis
import sys
import pandas as pd
sys.path.append("..")
from rcn_py import database
from crossref.restful import Works
from neo4j import GraphDatabase

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



    author-paper table
    author info table
    publication table

In [12]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

In [12]:
cur.execute("CREATE TABLE test_paper(doi primary key, title, abstract)")

In [13]:
cur.execute("CREATE TABLE test_authors(orcid primary key, name, country)")


In [16]:
cur.execute("CREATE TABLE test_author_papers2(orcid, doi, primary key(orcid, doi) FOREIGN KEY(orcid) REFERENCES test_authors(orcid), FOREIGN KEY(doi) REFERENCES test_paper(doi))")

In [55]:
con.commit()

In [56]:
con.close()

## Insert data

In [72]:
database.insert_database("/Users/jennifer/RCSNA/data/tutorial.db", '',"Peter Kalverla")

'Done.'

In [ ]:
database.insert_coauthors_pub("/Users/jennifer/RCSNA/data/tutorial.db", "Peter Kalverla")

In [ ]:
database.insert_cocoauthors("/Users/jennifer/RCSNA/data/tutorial.db")

### Fetch data from database

In [ ]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

# Topic modeling (title or abstract) to cluster papers
clusters, idx2topics = database.pub_cluster(cur)

In [ ]:
# get coauthor relationship
links = database.fetch_relationships(cur)

In [ ]:
# assign author's group as the most frequent paper group
orcid_group = database.author_cluster(cur, clusters)

cur.close()

In [ ]:
# pyvis
database.build_network_database("Database")

### Store more data

In [4]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

In [6]:
con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
cur_rsd = con_rsd.cursor()

In [18]:
res_rsd = cur_rsd.execute("""
SELECT orcid, given_name, family_name, affiliation 
FROM author 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
UNION
SELECT orcid, given_name, family_name, affiliation 
FROM contributor 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
""")
escience_member = res_rsd.fetchall()

In [25]:
escience_member[0]


('0000-0001-5301-3276',
 'Jesus',
 'Garcia Gonzalez',
 'Netherlands eScience Center')

In [2]:
def insert_escience_rsd_member(db_path):
    con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
    cur_rsd = con_rsd.cursor()
    res_rsd = cur_rsd.execute("""
        SELECT orcid, given_name, family_name, affiliation 
        FROM author 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        UNION
        SELECT orcid, given_name, family_name, affiliation 
        FROM contributor 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        """)
    escience_members = res_rsd.fetchall()
    for member in escience_members:
        orcid_id = member[0]
        fullname = member[1] + " " + member[2]
        database.insert_database(db_path, orcid_id, fullname)

    con_rsd.commit()
    con_rsd.close()
    return "Insertion complete"
    

In [3]:
insert_escience_rsd_member("/Users/jennifer/RCSNA/data/tutorial.db")

'Insertion complete'

In [14]:
# cur.execute("DELETE FROM author_publication WHERE doi = 'None'")
con.commit()
con.close()

## Neo4j

In [6]:
def get_pub_info(tx, df, subject):
    for i in range(len(df)):
        doi = df.DOI[i]
        title = df.Title[i]
        year = df.Year[i]
        cited = df["Cited by"][i]
        # subject = subject
        if not isinstance(df['Author Keywords'][i], str):
            keywords = []
        else:
            keywords = df["Author Keywords"][i].split("; ")
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        author_name = df["Authors"][i].split(", ")[0:len(author_scopus_id)]
        author_aff = df['Authors with affiliations'][i].split("; ")[0:len(author_scopus_id)]
        author_country = [aff.split(", ")[-1] for aff in author_aff]

        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue

        # APOC plugin should be installed in your Neo4j Server
        # Create people nodes
        for n in range(len(author_scopus_id)):
            # if the person exists, append keywords and year
            # avoid adding duplicate years
            tx.run("""
                MERGE (p:Person {scopus_id: $id})
                SET p.name = $name,
                    p.affiliation = $affiliation, 
                    p.country = $country,
                    p.keywords = coalesce(p.keywords, []) + $keywords,
                    p.year = apoc.coll.toSet(coalesce(p.year, []) + $year),
                    p.subject = coalesce(p.subject, []) + $subject
                """, 
                id = author_scopus_id[n],
                name = author_name[n],
                affiliation = author_aff[n],
                country = author_country[n],
                keywords = keywords,
                year = year,
                subject = subject
                )
            
        # Create edges
        for i in range(len(author_scopus_id))[0:-1]:
            for j in range(len(author_scopus_id))[i+1:]:
                # if the edge exists, add weight, doi, title, year
                # avoid adding duplicate years
                tx.run("""
                    MATCH 
                        (a:Person), 
                        (b:Person)
                    WHERE a.scopus_id = $person_a AND b.scopus_id = $person_b
                    MERGE (a)-[r:COAUTHOR]-(b)
                    SET r.doi = coalesce(r.doi, []) + $doi,
                        r.title = coalesce(r.title, []) + $title,
                        r.year = apoc.coll.toSet(a.year + b.year),
                        r.weight = size(r.doi)+1,
                        r.cited = coalesce(r.cited, []) + $cited,
                        r.subject = apoc.coll.toSet(coalesce(r.subject, []) + $subject)
                    """, 
                    person_a=author_scopus_id[i], 
                    person_b=author_scopus_id[j], 
                    doi = doi,
                    title = title,
                    cited = cited,
                    subject = subject
                    )

        

In [7]:
def neo4j_create_people(tx, df, subject):
    for i in range(len(df)):
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue
        year = df.Year[i]
        author_name = df["Authors"][i].split(", ")[0:len(author_scopus_id)]
        author_aff = df['Authors with affiliations'][i].split("; ")[0:len(author_scopus_id)]
        author_country = [aff.split(", ")[-1] for aff in author_aff]
        if not isinstance(df['Author Keywords'][i], str):
            keywords = []
        else:
            keywords = df["Author Keywords"][i].split("; ")

        # APOC plugin should be installed in your Neo4j Server
        # Create people nodes
        for n in range(len(author_scopus_id)):
            # if the person exists, append keywords and year
            # avoid adding duplicate years
            tx.run("""
                MERGE (p:Person {scopus_id: $id})
                SET p.name = $name,
                    p.affiliation = $affiliation, 
                    p.country = $country,
                    p.keywords = apoc.coll.toSet(coalesce(p.keywords, []) + $keywords),
                    p.year = apoc.coll.toSet(coalesce(p.year, []) + $year),
                    p.subject = apoc.coll.toSet(coalesce(p.subject, []) + $subject)
                """, 
                id = author_scopus_id[n],
                name = author_name[n],
                affiliation = author_aff[n],
                country = author_country[n],
                keywords = keywords,
                year = year,
                subject = subject
                )
        

In [33]:
def neo4j_create_publication(tx, df, subject):
    for i in range(len(df)):
        
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue

        doi = df.DOI[i]
        title = df.Title[i]
        year = df.Year[i]
        cited = df["Cited by"][i]
        # subject = subject
        if not isinstance(df['Author Keywords'][i], str):
            keywords = []
        else:
            keywords = df["Author Keywords"][i].split("; ")
        
        author_name = df["Authors"][i].split(", ")[0:len(author_scopus_id)]
        author_aff = df['Authors with affiliations'][i].split("; ")[0:len(author_scopus_id)]
        author_country = [aff.split(", ")[-1] for aff in author_aff]

        # Create publication nodes
        tx.run("""
                MERGE (p:Publication {doi: $doi})
                SET p.title = $title,
                    p.year = $year, 
                    p.cited = $cited,
                    p.keywords = $keywords,
                    p.subject = apoc.coll.toSet(coalesce(p.subject, []) + $subject)
                """, 
                doi = doi,
                title = title,
                year = year,
                cited = cited,
                keywords = keywords,
                subject = subject
                )
        
        

In [9]:
def neo4j_create_author_pub_edge(tx, df):
    for i in range(len(df)):
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue     

        author_name = df["Authors"][i].split(", ")[0:len(author_scopus_id)]   
        year = df.Year[i]
        doi = df.DOI[i]
        title = df.Title[i]
        # APOC plugin should be installed in your Neo4j Server
        # Create edges
        for i in range(len(author_scopus_id)):
            tx.run("""
                    MATCH 
                        (n:Person {scopus_id: $person_id}), 
                        (p:Publication {doi: $doi})
                    MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                    SET r.year = $year,
                        r.author_name = $author_name,
                        r.title = $title
                    """, 
                    person_id = author_scopus_id[i], 
                    doi = doi, 
                    year = year,
                    author_name = author_name[i],
                    title = title
                    )

In [10]:
def neo4j_create_relationship(tx, df, subject):
    for i in range(len(df)):
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue        

        doi = df.DOI[i]
        title = df.Title[i]
        cited = df["Cited by"][i]

        # APOC plugin should be installed in your Neo4j Server
        # Create edges
        for i in range(len(author_scopus_id))[0:-1]:
            for j in range(len(author_scopus_id))[i+1:]:
                # if the edge exists, add weight, doi, title, year
                # avoid adding duplicate years
                tx.run("""
                    MATCH 
                        (a:Person {scopus_id: $person_a}), 
                        (b:Person {scopus_id: $person_b})
                    MERGE (a)-[r:COAUTHOR]-(b)
                    SET r.doi = coalesce(r.doi, []) + $doi,
                        r.title = coalesce(r.title, []) + $title,
                        r.year = apoc.coll.toSet(a.year + b.year),
                        r.weight = size(r.doi)+1,
                        r.cited = coalesce(r.cited, []) + $cited,
                        r.subject = apoc.coll.toSet(coalesce(r.subject, []) + $subject)
                    """, 
                    person_a=author_scopus_id[i], 
                    person_b=author_scopus_id[j], 
                    doi = doi,
                    title = title,
                    cited = cited,
                    subject = subject
                    )
        

In [ ]:
# remote AuraDB, only one for free'
# uri = "neo4j+s://a93e1a64.databases.neo4j.io"
# user = "neo4j"
# password = "ifVwf-BGTZt1ltrNgVbh5Q6aGSxQSksQJSxBUbJb1BE"

In [36]:
# local AuraDB
uri = "bolt://localhost:7687"
user = "neo4j"
password = "zhiningbai"

In [38]:
check_verify =  GraphDatabase.driver(uri, auth=(user, password))
check_verify.verify_connectivity()

In [29]:
nl_medi_23 = pd.read_csv("/Users/jennifer/scopus_data/year22/nl_medicine1.csv")
nl_medi_23[0:1]

,Authors,Author(s) ID,Title,Year,Cited by,DOI,Link,Affiliations,Authors with affiliations,Author Keywords
0,"Wang Z., Lorenzut G., Zhang Z., Dekker A., Tra...",57208488517;58094235900;57295928000;5722537918...,Applications of generative adversarial network...,2022,NaN,10.21037/pcm-22-28,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Radiation Oncology (Maastro), GR...","Wang, Z., Department of Radiation Oncology (Ma...",applications; Generative Adversarial Network (...


In [39]:
# Execution
# Filepath and subject depends on the developer's data
for i in range(1,12):
    path = "/Users/jennifer/scopus_data/year22/nl_medicine"+str(i)+".csv"
    df = pd.read_csv(path)
    subject = "Medicine"

    with GraphDatabase.driver(uri, auth=(user, password)) as driver:
        driver.verify_connectivity()
        with driver.session(database="neo4j") as session:
            # Create nodes & edges
            session.execute_write(neo4j_create_people, df, subject) 
            session.execute_write(neo4j_create_publication, df, subject)
            session.execute_write(neo4j_create_author_pub_edge, df)
    print ("nl_medicine"+str(i)+".csv is Done!")      
            
            

nl_medicine1.csv is Done!
nl_medicine2.csv is Done!
nl_medicine3.csv is Done!
nl_medicine4.csv is Done!
nl_medicine5.csv is Done!
nl_medicine6.csv is Done!
nl_medicine7.csv is Done!
nl_medicine8.csv is Done!
nl_medicine9.csv is Done!
nl_medicine10.csv is Done!
nl_medicine11.csv is Done!


In [ ]:
# Execution
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    with driver.session(database="neo4j") as session:
        # Create nodes & edges
        for i in range(1,5):
            path = "/Users/jennifer/scopus_data/year22/nl_bioc"+str(i)+".csv"
            df = pd.read_csv(path)
            subject = "Biochemistry, Genetics and Molecular Biology"
       
            session.execute_write(neo4j_create_people, df, subject) 
            session.execute_write(neo4j_create_publication, df, subject)
            session.execute_write(neo4j_create_author_pub_edge, df)
            print (subject+str(i)+" is Done!")      
            

In [40]:
session.close()
driver.close()